##**4. LSTM, GRU**
1. 기존 RNN과 다른 부분에 대해서 배웁니다.
2. 이전 실습에 이어 다양한 적용법을 배웁니다.

### **필요 패키지 import**

In [5]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch

### **데이터 전처리**

아래의 sample data를 확인해봅시다.  
이전 실습과 동일합니다.

In [6]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [7]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 54755.93it/s]


In [8]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

### **LSTM 사용**

LSTM에선 cell state가 추가됩니다.  
Cell state의 shape는 hidden state의 그것과 동일합니다.

In [10]:
embedding_size = 256
hidden_size = 512
num_layers = 1
num_dirs = 1

embedding = nn.Embedding(vocab_size, embedding_size)
# ---------------------fill in ------------------------
lstm = nn.LSTM(
    input_size = embedding_size,
    hidden_size = hidden_size, # hidden_state와 같은 size를 가진다
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False

)

h_0 = torch.zeros(num_layers * num_dirs, batch.shape[0], hidden_size)  # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros(num_layers * num_dirs, batch.shape[0], hidden_size)  # (num_layers * num_dirs, B, d_h)
# ---------------------fill in ------------------------

In [12]:
# d_w: word embedding size
batch_emb = embedding(batch)  # (B, L, d_w)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, (h_n, c_n) = lstm(packed_batch, (h_0, c_0))

print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)
print(c_n.shape)

PackedSequence(data=tensor([[ 0.1824,  0.0775,  0.0523,  ...,  0.0008, -0.1086, -0.0799],
        [ 0.0194,  0.0365, -0.0320,  ...,  0.1192,  0.1114, -0.0124],
        [ 0.1333, -0.0247,  0.1339,  ..., -0.0887,  0.0798, -0.0313],
        ...,
        [-0.0979,  0.0989, -0.0329,  ..., -0.0021, -0.0887,  0.1320],
        [-0.1188, -0.0269,  0.0088,  ...,  0.0887,  0.0488,  0.0991],
        [-0.0795, -0.0683,  0.0425,  ..., -0.0937,  0.0678,  0.0728]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])


In [13]:
outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)
print(output_lens)

torch.Size([20, 10, 512])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


### **GRU 사용**

GRU는 cell state가 없어 RNN과 동일하게 사용 가능합니다.   
GRU를 이용하여 LM task를 수행해봅시다.

In [16]:
num_layers = 1
num_dirs = 1
dropout=0.1

# ---------------------fill in ------------------------
gru = nn.GRU(
    input_size = embedding_size,
    hidden_size = hidden_size,
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False,
    dropout = dropout
)
# ---------------------fill in ------------------------

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [17]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [18]:
input_id = batch.transpose(0, 1)[0, :]  # (B)
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

Teacher forcing 없이 이전에 얻은 결과를 다음 input으로 이용합니다.

In [19]:
for t in range(max_len):
# ---------------------fill in ------------------------
  input_emb = embedding(input_id).unsqueeze(0) # (B, embedding_size)
  output, hidden = gru(input_emb, hidden) # output: (1, B, 2d_h), hidden: (1, B, d_h)

  # V: vocab size
  output = output_layer(output) # (1, B, V)
  probs, top_id = torch.max(output, dim=-1) # use torch.max # probs: (1, B), top_id: (1, B)
# ---------------------fill in ------------------------

  print("*" * 50)
  print(f"Time step: {t}")
  print(output.shape)
  print(probs.shape)
  print(top_id.shape)

  input_id = top_id.squeeze(0)  # (B)

**************************************************
Time step: 0
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 1
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 2
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 3
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 4
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 5
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 6
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 7
torch.Size([1, 10, 100])
torch.Si

`max_len`만큼의 for 문을 돌면서 모든 결과물의 모양을 확인했지만 만약 종료 조건(예를 들어 문장의 끝을 나타내는 end token 등)이 되면 중간에 생성을 그만둘 수도 있습니다.

### **양방향 및 여러 layer 사용**

이번엔 양방향 + 2개 이상의 layer를 쓸 때 얻을 수 있는 결과에 대해 알아봅니다.

In [20]:
num_layers = 2
num_dirs = 2
dropout=0.1

# ---------------------fill in ------------------------
gru = nn.GRU(
    input_size = embedding_size,
    hidden_size = hidden_size,
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False,
    dropout = dropout
)
# ---------------------fill in ------------------------

Bidirectional이 되었고 layer의 개수가 $2$로 늘었기 때문에 hidden state의 shape도 `(4, B, d_h)`가 됩니다.

In [21]:
# d_w: word embedding size, num_layers: layer의 개수, num_dirs: 방향의 개수
batch_emb = embedding(batch)  # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

PackedSequence(data=tensor([[ 0.0086,  0.0444,  0.2085,  ..., -0.1152, -0.0325,  0.0265],
        [ 0.1405, -0.0113,  0.0971,  ..., -0.1391, -0.1674, -0.0039],
        [-0.0206,  0.1441,  0.1972,  ..., -0.2586, -0.0901, -0.1298],
        ...,
        [ 0.3757,  0.0081,  0.0317,  ..., -0.0737,  0.0219,  0.0068],
        [ 0.3367, -0.1520,  0.1457,  ..., -0.0227,  0.0905, -0.0835],
        [ 0.3294, -0.0961,  0.1549,  ...,  0.1122, -0.0427, -0.0427]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 1024])
torch.Size([4, 10, 512])


In [22]:
outputs, output_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

torch.Size([20, 10, 1024])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


각각의 결과물의 shape는 다음과 같습니다.

`outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
`h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [23]:
batch_size = h_n.shape[1]
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)

tensor([[[[-0.2619, -0.1103,  0.0929,  ...,  0.2554,  0.0517,  0.2495],
          [ 0.0665,  0.2286,  0.1908,  ..., -0.0792,  0.0275,  0.1626],
          [ 0.0316,  0.1185,  0.0387,  ...,  0.2256, -0.2847,  0.3328],
          ...,
          [-0.0815, -0.2650,  0.1230,  ..., -0.0883,  0.2022,  0.0359],
          [ 0.2881, -0.2239,  0.1764,  ...,  0.3514,  0.1189, -0.0349],
          [-0.1226, -0.0583, -0.3923,  ...,  0.0463,  0.3502,  0.3457]],

         [[ 0.1995, -0.3320,  0.1185,  ...,  0.0458, -0.1279, -0.0207],
          [ 0.4960, -0.3557,  0.2467,  ..., -0.2606, -0.1816, -0.0065],
          [-0.0051, -0.3883,  0.1425,  ..., -0.2242, -0.0843, -0.1228],
          ...,
          [-0.3959,  0.2148, -0.1465,  ...,  0.1105,  0.0083, -0.1758],
          [ 0.0314,  0.0335,  0.3389,  ..., -0.2660, -0.3951,  0.1600],
          [ 0.0490, -0.0481, -0.0477,  ..., -0.3762,  0.2735, -0.1161]]],


        [[[ 0.3294, -0.0961,  0.1549,  ..., -0.0759,  0.3469,  0.1844],
          [-0.1412,  0.1194,